In [2]:
!nvidia-smi

Tue Dec 15 08:12:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   58C    P2   237W / 280W |  12975MiB / 24220MiB |     65%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 66%   85C    P2   257W / 280W |  14370MiB / 24220MiB |     62%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torch.optim as optim
from torchvision import models


sys.path.append('/home/ironman/abhishek/AliProducts/architectures/')
sys.path.append('/home/ironman/abhishek/AliProducts/Helper/')
from dataloader import mydataset, create_prime_dict 
from trainer_cutmix import train, test_classify, eval_classify
from resnet_models import ResNet,Bottleneck
from Load_model import load
from plot_curves import plot_loss, plot_acc


from collections import defaultdict

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Dataloading Scheme**

In [5]:
trainlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_train_list1.txt'
validlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_valid_list1.txt'

In [6]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  50030


In [7]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 64, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

**Model Definition**

In [8]:
model = models.resnext101_32x8d(pretrained = False, num_classes = 50030)

model = nn.DataParallel(model,device_ids=[0,2,3,5]).to(device)
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

**Hyperparameters**

In [9]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 25, gamma = 0.1)

# Epochs
num_Epochs = 90

beta=1

cutmix_prob = 1

In [10]:
modelname = 'NewData_Apro_AA_Cutmix_ResNeXt101_32x8d'
modelpath = '/home/ironman/abhishek/saved_model_checkpoints/AliProducts/'+modelname

In [11]:
#### Load saved model from checkpoint  #####
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [ ]:
t_loss,top1_t_acc, top5_t_acc =test_classify(model, train_dataloader, criterion, device)
print('Training Loss: {:.4f}\tTop 1 Training Accuracy: {:.4f}\t Top 5 Training Accuracy: {:.4f}'.format(t_loss, top1_t_acc, top5_t_acc))

In [42]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

KeyboardInterrupt: 

In [12]:
def inverse_prime_dict(prime_dict):
    
    inverse_prime_dict = {}
    
    for key in prime_dict.keys():
        
        inverse_prime_dict[prime_dict[key]] = key
    
    return inverse_prime_dict

inverse_dict = inverse_prime_dict(prime_dict) 


In [13]:
'''
Error Analysis

misclassified into which class

error_dict: classes: misclassified class: number of times

error_dict_count: classes:number of times misclassified
'''


def error_analysis(model, test_loader, criterion, inverse_dict, device):
    model.eval()
    test_loss = []
    top1_accuracy = 0
    top5_accuracy = 0
    total = 0
    
    error_dict = defaultdict(lambda:defaultdict(int))
    error_dict_count = defaultdict(int)

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats) 
        
        '''
        find loss
        '''
        loss = criterion(outputs, labels.long())
        test_loss.extend([loss.item()]*feats.size()[0])
        
        
        '''
        Predictions
        '''
        predictions = F.softmax(outputs, dim=1)
                
            
        '''
        Top1 Accuracy
        '''
        _, top1_pred_labels = torch.max(predictions,1)
        top1_pred_labels = top1_pred_labels.view(-1)
        
        top1_accuracy += torch.sum(torch.eq(top1_pred_labels, labels)).item()
        
        
        '''
        Misclassified instances
        '''
        
        for i in range(len(labels)):

            if not torch.eq(labels[i],top1_pred_labels[i]):
                
                actual = inverse_dict[labels[i].item()]
                predicted = inverse_dict[top1_pred_labels[i].item()]
                
                error_dict[actual][predicted]+=1
                error_dict_count[actual]+=1
                


                
        total += len(labels)
        
        
    model.train()
    return np.mean(test_loss), top1_accuracy/total, error_dict, error_dict_count

In [14]:
loss, top1, error_dict, error_dict_count = error_analysis(model, train_dataloader, criterion, inverse_dict, device)

In [36]:
error_dict['41662']

defaultdict(int, {'15634': 3, '43535': 9, '867': 4})

In [21]:
def sorted_dict(dictionary):
    
    sorted_keys = sorted(dictionary, key =dictionary.__getitem__, reverse=True)
    sorted_values = sorted(dictionary.values(),reverse=True)
    
    sorted_dictionary = {}
    for i in range(len(sorted_keys)):
        sorted_dictionary[sorted_keys[i]] = sorted_values[i]
        
    return sorted_dictionary

In [22]:
sorted_dict(error_dict_count)

{'45589': 27,
 '20220': 27,
 '11807': 24,
 '11835': 24,
 '23328': 24,
 '6644': 23,
 '18122': 22,
 '49442': 22,
 '2540': 22,
 '11832': 22,
 '9724': 22,
 '34673': 22,
 '22083': 22,
 '25937': 21,
 '7908': 21,
 '11865': 21,
 '32035': 21,
 '2334': 21,
 '22128': 21,
 '22130': 21,
 '22030': 21,
 '18793': 21,
 '22525': 21,
 '21975': 20,
 '24756': 20,
 '30447': 20,
 '7015': 20,
 '8273': 20,
 '11617': 20,
 '9627': 20,
 '26460': 20,
 '11839': 20,
 '4487': 20,
 '11916': 20,
 '13941': 20,
 '38622': 20,
 '38889': 20,
 '407': 20,
 '21572': 20,
 '24256': 20,
 '26682': 20,
 '19663': 19,
 '46918': 19,
 '15915': 19,
 '26192': 19,
 '20383': 19,
 '13239': 19,
 '43272': 19,
 '7467': 19,
 '38268': 19,
 '39753': 19,
 '38162': 19,
 '15585': 19,
 '16451': 19,
 '21339': 19,
 '29601': 19,
 '14084': 19,
 '20167': 19,
 '12370': 19,
 '13258': 19,
 '6589': 19,
 '42817': 18,
 '35468': 18,
 '39137': 18,
 '22180': 18,
 '9937': 18,
 '31585': 18,
 '49532': 18,
 '8318': 18,
 '11456': 18,
 '29251': 18,
 '18147': 18,
 '16924

In [18]:
'''
See where error is coming from, i.e from classes <30 samples or from more than 30 samples
represented_dict- whether a class is higly represented or less represented (more than 30 samples or not)
'''

'''
load classes
'''
f = open('classes_more_than_30_images_list','r')
highly_represented =f.readlines()
f.close()
highly_represented = [i[:-1] for i in highly_represented]    

f = open('classes_less_than_30_images_list','r')
less_represented =f.readlines()
f.close()
less_represented=[i[:-1] for i in less_represented]

represented_dict = defaultdict(int)

for key in error_dict_count:
    if key in highly_represented:
        represented_dict['highly represented'] +=1
    else:
        represented_dict['less represented'] +=1
    

In [19]:
represented_dict

defaultdict(int, {'highly represented': 12180, 'less represented': 15993})